# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [234]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [235]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [236]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    vocab = set(text)
    vocab_to_int = {c: i for i, c in enumerate(vocab)}
    int_to_vocab = dict(enumerate(vocab))
    print(vocab_to_int)
    return vocab_to_int, int_to_vocab
    


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

{'barney_gumble': 0, 'mike': 1, 'lately': 2, 'last': 3, 'little': 6, 'hold': 5, 'homer_simpson': 10, 'effervescent': 8, 'yeah': 9, 'eh': 27, 'moe_szyslak': 12, 'gonna': 13, 'hey': 14, 'tavern': 15, "you're": 16, 'where': 17, 'your': 18, 'drink': 19, 'i': 20, 'skills': 22, 'check': 4, 'problems': 23, 'give': 24, 'should': 25, 'social': 64, 'homer': 26, 'of': 28, 'there': 52, 'listen': 29, 'ice': 32, 'one': 65, 'my': 36, 'and': 34, "i'm": 35, 'enhance': 30, 'not': 60, 'on': 33, 'meet': 37, 'elite': 38, 'got': 40, 'you': 41, 'only': 11, 'puke': 42, 'is': 43, 'forget': 21, 'days': 46, 'anybody': 47, 'to': 49, 'rotch': 50, 'with': 39, 'carve': 44, 'bart_simpson': 51, 'these': 54, 'the': 53, 'self': 7, 'matter': 55, 'an': 48, 'seen': 56, 'another': 57, 'hello': 58, 'back': 59, 'whats': 61, 'catch': 62, "moe's": 63, 'name': 31, 'normal': 45, 'pick': 66, 'me': 67, 'moe': 68, "i'll": 69, 'has': 70}
Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [237]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    values = ['||Period||', '||Comma||', '||Quotation_Mark||', '||Semicolon||', '||Exclamation_Mark||', '||Question_Mark||', \
              '||Left_Parentheses||', '||Right_Parentheses||', '||Dash||', '||Return||']
    keys = ['.', ',', '"', ';', '!', '?', '(', ')', '--', '\n']
    return (dict(zip(keys, values)))

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [238]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

{'mommy': 3477, 'roz': 5823, 'bow': 0, 'hitchhike': 5110, 'we': 1197, 'helping': 1, 'tiny': 2, 'aisle': 3478, 'tatum': 3479, 'grammar': 3, 'customer': 3480, 'pressure': 4, "y'see": 5, 'housewife': 6, 'admiration': 8, 'bathtub': 3483, 'gargoyle': 9, 'cleaned': 3484, 'international': 3486, 'carl': 10, 'classy': 11, 'coherent': 3488, 'tip': 12, 'statue': 13, 'sink': 14, 'jägermeister': 601, 'since': 15, 'timbuk-tee': 17, 'sex': 600, 'divine': 3492, 'mediterranean': 3493, 'attractive_woman_#2:': 18, 'flack': 3494, 'buddha': 3495, 'quimby_#2:': 19, 'size': 605, "burnin'": 3496, 'skins': 3497, 'pointless': 3498, 'put': 20, 'rain': 21, 'zoomed': 22, "sippin'": 1083, 'prolonged': 5930, 'telephone': 3500, 'caveman': 3501, 'term': 3503, 'whenever': 3504, 'convenient': 3505, 'recall': 3506, 'childless': 608, 'yours': 23, 'crowded': 1203, 'hardwood': 5584, 'bidet': 24, 'stalwart': 3507, 'enemies': 3508, 'david_byrne:': 25, 'practically': 3510, 'corporate': 26, '&': 27, 'device': 3511, 'low-blow': 

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [239]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [240]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.1
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [241]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    input = tf.placeholder(tf.int32, (None, None), 'input')
    targets = tf.placeholder(tf.int32, (None, None), 'targets')
    learning_rate = tf.placeholder(tf.float32, None, 'learning_rate')
#     print(input)
#     print(targets)
    return input, targets, learning_rate


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [242]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    number_of_layers = 2
    cell = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    cell = tf.contrib.rnn.MultiRNNCell([cell]*number_of_layers)
    
    X = cell.zero_state(batch_size, tf.float32)
    initialize_state = tf.identity(X, name='initial_state')
    
    return cell, initialize_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [243]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
#     print(input_data)
#     print(vocab_size)
#     print(embed_dim)
#     graph = tf.Graph()
#     with graph.as_default():
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim)))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [244]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    
    (outputs, FinalState) = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    FinalState = tf.identity(FinalState, name="final_state")
    return outputs, FinalState


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [245]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
#     print(rnn_size)
#     print(input_data)
#     print(vocab_size)
#     print(embed_dim)
    inputs = get_embed(input_data, vocab_size, 200)
    outputs, FinalState = build_rnn(cell, inputs)
    logits = tf.contrib.layers.fully_connected(outputs, vocab_size, activation_fn=None)
    return logits, FinalState


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [246]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    batches = batch_size * seq_length
    batch_number = len(int_text)//(batches)
#     print ("seg_length:", seq_length)
#     print("batch_number:", batch_number)
#     print("batch_size:", batch_size)
#     print("batches:", batches)
#     print("text replaced by id:", int_text)
    x = np.array(int_text[: batch_number * batch_size * seq_length])
    y = np.array(int_text[1: batch_number * batch_size * seq_length + 1])
    y[-1] = x[0]
    x_batches = np.split(x.reshape(batch_size, -1), batch_number, 1)
    y_batches = np.split(y.reshape(batch_size, -1), batch_number, 1)
    return np.array(list(zip(x_batches, y_batches)))


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [247]:
# Number of Epochs
num_epochs = 300
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 256
# Embedding Dimension Size
embed_dim = 300
# Sequence Length
seq_length = 5
# Learning Rate
learning_rate = .01
# Show stats for every n number of batches
show_every_n_batches = 2

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [248]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [249]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/107   train_loss = 8.821
Epoch   0 Batch    2/107   train_loss = 7.006
Epoch   0 Batch    4/107   train_loss = 6.466
Epoch   0 Batch    6/107   train_loss = 6.755
Epoch   0 Batch    8/107   train_loss = 7.316
Epoch   0 Batch   10/107   train_loss = 6.811
Epoch   0 Batch   12/107   train_loss = 7.020
Epoch   0 Batch   14/107   train_loss = 6.962
Epoch   0 Batch   16/107   train_loss = 7.241
Epoch   0 Batch   18/107   train_loss = 7.118
Epoch   0 Batch   20/107   train_loss = 7.279
Epoch   0 Batch   22/107   train_loss = 7.237
Epoch   0 Batch   24/107   train_loss = 7.329
Epoch   0 Batch   26/107   train_loss = 7.208
Epoch   0 Batch   28/107   train_loss = 6.440
Epoch   0 Batch   30/107   train_loss = 6.600
Epoch   0 Batch   32/107   train_loss = 7.031
Epoch   0 Batch   34/107   train_loss = 6.892
Epoch   0 Batch   36/107   train_loss = 6.651
Epoch   0 Batch   38/107   train_loss = 7.081
Epoch   0 Batch   40/107   train_loss = 7.251
Epoch   0 Batch   42/107   train_l

Epoch   3 Batch   43/107   train_loss = 6.244
Epoch   3 Batch   45/107   train_loss = 6.304
Epoch   3 Batch   47/107   train_loss = 6.201
Epoch   3 Batch   49/107   train_loss = 6.331
Epoch   3 Batch   51/107   train_loss = 6.456
Epoch   3 Batch   53/107   train_loss = 6.263
Epoch   3 Batch   55/107   train_loss = 6.426
Epoch   3 Batch   57/107   train_loss = 6.251
Epoch   3 Batch   59/107   train_loss = 6.226
Epoch   3 Batch   61/107   train_loss = 6.303
Epoch   3 Batch   63/107   train_loss = 6.247
Epoch   3 Batch   65/107   train_loss = 6.346
Epoch   3 Batch   67/107   train_loss = 6.241
Epoch   3 Batch   69/107   train_loss = 6.384
Epoch   3 Batch   71/107   train_loss = 6.309
Epoch   3 Batch   73/107   train_loss = 6.442
Epoch   3 Batch   75/107   train_loss = 6.217
Epoch   3 Batch   77/107   train_loss = 6.201
Epoch   3 Batch   79/107   train_loss = 6.226
Epoch   3 Batch   81/107   train_loss = 6.214
Epoch   3 Batch   83/107   train_loss = 6.007
Epoch   3 Batch   85/107   train_l

Epoch   6 Batch   82/107   train_loss = 5.932
Epoch   6 Batch   84/107   train_loss = 5.974
Epoch   6 Batch   86/107   train_loss = 5.789
Epoch   6 Batch   88/107   train_loss = 6.100
Epoch   6 Batch   90/107   train_loss = 5.990
Epoch   6 Batch   92/107   train_loss = 5.999
Epoch   6 Batch   94/107   train_loss = 5.987
Epoch   6 Batch   96/107   train_loss = 5.981
Epoch   6 Batch   98/107   train_loss = 5.881
Epoch   6 Batch  100/107   train_loss = 5.914
Epoch   6 Batch  102/107   train_loss = 6.023
Epoch   6 Batch  104/107   train_loss = 5.942
Epoch   6 Batch  106/107   train_loss = 5.934
Epoch   7 Batch    1/107   train_loss = 5.968
Epoch   7 Batch    3/107   train_loss = 5.802
Epoch   7 Batch    5/107   train_loss = 5.685
Epoch   7 Batch    7/107   train_loss = 5.639
Epoch   7 Batch    9/107   train_loss = 5.797
Epoch   7 Batch   11/107   train_loss = 5.781
Epoch   7 Batch   13/107   train_loss = 5.716
Epoch   7 Batch   15/107   train_loss = 5.906
Epoch   7 Batch   17/107   train_l

Epoch  10 Batch   14/107   train_loss = 4.613
Epoch  10 Batch   16/107   train_loss = 4.906
Epoch  10 Batch   18/107   train_loss = 4.738
Epoch  10 Batch   20/107   train_loss = 4.847
Epoch  10 Batch   22/107   train_loss = 4.994
Epoch  10 Batch   24/107   train_loss = 4.820
Epoch  10 Batch   26/107   train_loss = 5.041
Epoch  10 Batch   28/107   train_loss = 4.510
Epoch  10 Batch   30/107   train_loss = 4.666
Epoch  10 Batch   32/107   train_loss = 4.696
Epoch  10 Batch   34/107   train_loss = 4.884
Epoch  10 Batch   36/107   train_loss = 4.727
Epoch  10 Batch   38/107   train_loss = 4.769
Epoch  10 Batch   40/107   train_loss = 4.980
Epoch  10 Batch   42/107   train_loss = 4.762
Epoch  10 Batch   44/107   train_loss = 4.639
Epoch  10 Batch   46/107   train_loss = 4.817
Epoch  10 Batch   48/107   train_loss = 4.611
Epoch  10 Batch   50/107   train_loss = 4.568
Epoch  10 Batch   52/107   train_loss = 4.747
Epoch  10 Batch   54/107   train_loss = 4.653
Epoch  10 Batch   56/107   train_l

Epoch  13 Batch   53/107   train_loss = 4.474
Epoch  13 Batch   55/107   train_loss = 4.638
Epoch  13 Batch   57/107   train_loss = 4.460
Epoch  13 Batch   59/107   train_loss = 4.550
Epoch  13 Batch   61/107   train_loss = 4.523
Epoch  13 Batch   63/107   train_loss = 4.530
Epoch  13 Batch   65/107   train_loss = 4.594
Epoch  13 Batch   67/107   train_loss = 4.544
Epoch  13 Batch   69/107   train_loss = 4.439
Epoch  13 Batch   71/107   train_loss = 4.602
Epoch  13 Batch   73/107   train_loss = 4.723
Epoch  13 Batch   75/107   train_loss = 4.267
Epoch  13 Batch   77/107   train_loss = 4.545
Epoch  13 Batch   79/107   train_loss = 4.455
Epoch  13 Batch   81/107   train_loss = 4.460
Epoch  13 Batch   83/107   train_loss = 4.252
Epoch  13 Batch   85/107   train_loss = 4.421
Epoch  13 Batch   87/107   train_loss = 4.481
Epoch  13 Batch   89/107   train_loss = 4.718
Epoch  13 Batch   91/107   train_loss = 4.668
Epoch  13 Batch   93/107   train_loss = 4.558
Epoch  13 Batch   95/107   train_l

...

Epoch 286 Batch    2/107   train_loss = 0.878
Epoch 286 Batch    4/107   train_loss = 0.939
Epoch 286 Batch    6/107   train_loss = 0.936
Epoch 286 Batch    8/107   train_loss = 1.042
Epoch 286 Batch   10/107   train_loss = 0.969
Epoch 286 Batch   12/107   train_loss = 0.912
Epoch 286 Batch   14/107   train_loss = 0.935
Epoch 286 Batch   16/107   train_loss = 0.938
Epoch 286 Batch   18/107   train_loss = 0.965
Epoch 286 Batch   20/107   train_loss = 0.910
Epoch 286 Batch   22/107   train_loss = 0.979
Epoch 286 Batch   24/107   train_loss = 1.003
Epoch 286 Batch   26/107   train_loss = 0.922
Epoch 286 Batch   28/107   train_loss = 0.904
Epoch 286 Batch   30/107   train_loss = 0.929
Epoch 286 Batch   32/107   train_loss = 0.940
Epoch 286 Batch   34/107   train_loss = 0.936
Epoch 286 Batch   36/107   train_loss = 0.951
Epoch 286 Batch   38/107   train_loss = 0.912
Epoch 286 Batch   40/107   train_loss = 0.951
Epoch 286 Batch   42/107   train_loss = 0.972
Epoch 286 Batch   44/107   train_l

Epoch 289 Batch   41/107   train_loss = 0.855
Epoch 289 Batch   43/107   train_loss = 0.895
Epoch 289 Batch   45/107   train_loss = 0.850
Epoch 289 Batch   47/107   train_loss = 0.961
Epoch 289 Batch   49/107   train_loss = 0.967
Epoch 289 Batch   51/107   train_loss = 0.995
Epoch 289 Batch   53/107   train_loss = 0.913
Epoch 289 Batch   55/107   train_loss = 0.894
Epoch 289 Batch   57/107   train_loss = 1.002
Epoch 289 Batch   59/107   train_loss = 0.956
Epoch 289 Batch   61/107   train_loss = 0.912
Epoch 289 Batch   63/107   train_loss = 0.883
Epoch 289 Batch   65/107   train_loss = 0.889
Epoch 289 Batch   67/107   train_loss = 0.911
Epoch 289 Batch   69/107   train_loss = 0.898
Epoch 289 Batch   71/107   train_loss = 0.893
Epoch 289 Batch   73/107   train_loss = 0.902
Epoch 289 Batch   75/107   train_loss = 0.902
Epoch 289 Batch   77/107   train_loss = 0.893
Epoch 289 Batch   79/107   train_loss = 0.917
Epoch 289 Batch   81/107   train_loss = 0.895
Epoch 289 Batch   83/107   train_l

Epoch 292 Batch   80/107   train_loss = 0.959
Epoch 292 Batch   82/107   train_loss = 0.963
Epoch 292 Batch   84/107   train_loss = 0.944
Epoch 292 Batch   86/107   train_loss = 0.961
Epoch 292 Batch   88/107   train_loss = 1.017
Epoch 292 Batch   90/107   train_loss = 0.844
Epoch 292 Batch   92/107   train_loss = 0.898
Epoch 292 Batch   94/107   train_loss = 0.910
Epoch 292 Batch   96/107   train_loss = 0.934
Epoch 292 Batch   98/107   train_loss = 0.945
Epoch 292 Batch  100/107   train_loss = 0.835
Epoch 292 Batch  102/107   train_loss = 0.966
Epoch 292 Batch  104/107   train_loss = 0.836
Epoch 292 Batch  106/107   train_loss = 0.814
Epoch 293 Batch    1/107   train_loss = 0.931
Epoch 293 Batch    3/107   train_loss = 1.026
Epoch 293 Batch    5/107   train_loss = 0.890
Epoch 293 Batch    7/107   train_loss = 0.900
Epoch 293 Batch    9/107   train_loss = 0.884
Epoch 293 Batch   11/107   train_loss = 0.926
Epoch 293 Batch   13/107   train_loss = 0.939
Epoch 293 Batch   15/107   train_l

Epoch 296 Batch   12/107   train_loss = 0.957
Epoch 296 Batch   14/107   train_loss = 0.984
Epoch 296 Batch   16/107   train_loss = 0.991
Epoch 296 Batch   18/107   train_loss = 1.020
Epoch 296 Batch   20/107   train_loss = 0.973
Epoch 296 Batch   22/107   train_loss = 1.055
Epoch 296 Batch   24/107   train_loss = 1.038
Epoch 296 Batch   26/107   train_loss = 0.998
Epoch 296 Batch   28/107   train_loss = 0.956
Epoch 296 Batch   30/107   train_loss = 0.991
Epoch 296 Batch   32/107   train_loss = 0.987
Epoch 296 Batch   34/107   train_loss = 1.028
Epoch 296 Batch   36/107   train_loss = 1.014
Epoch 296 Batch   38/107   train_loss = 0.935
Epoch 296 Batch   40/107   train_loss = 1.062
Epoch 296 Batch   42/107   train_loss = 1.070
Epoch 296 Batch   44/107   train_loss = 0.981
Epoch 296 Batch   46/107   train_loss = 1.001
Epoch 296 Batch   48/107   train_loss = 1.085
Epoch 296 Batch   50/107   train_loss = 0.977
Epoch 296 Batch   52/107   train_loss = 0.989
Epoch 296 Batch   54/107   train_l

Epoch 299 Batch   51/107   train_loss = 1.533
Epoch 299 Batch   53/107   train_loss = 1.414
Epoch 299 Batch   55/107   train_loss = 1.521
Epoch 299 Batch   57/107   train_loss = 1.467
Epoch 299 Batch   59/107   train_loss = 1.521
Epoch 299 Batch   61/107   train_loss = 1.562
Epoch 299 Batch   63/107   train_loss = 1.459
Epoch 299 Batch   65/107   train_loss = 1.468
Epoch 299 Batch   67/107   train_loss = 1.425
Epoch 299 Batch   69/107   train_loss = 1.483
Epoch 299 Batch   71/107   train_loss = 1.601
Epoch 299 Batch   73/107   train_loss = 1.467
Epoch 299 Batch   75/107   train_loss = 1.537
Epoch 299 Batch   77/107   train_loss = 1.492
Epoch 299 Batch   79/107   train_loss = 1.537
Epoch 299 Batch   81/107   train_loss = 1.481
Epoch 299 Batch   83/107   train_loss = 1.495
Epoch 299 Batch   85/107   train_loss = 1.617
Epoch 299 Batch   87/107   train_loss = 1.569
Epoch 299 Batch   89/107   train_loss = 1.514
Epoch 299 Batch   91/107   train_loss = 1.528
Epoch 299 Batch   93/107   train_l

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [250]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [251]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [252]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    # with tf.Session() as sess:
    InputTensor = loaded_graph.get_tensor_by_name("input:0")
    InitialStateTensor = loaded_graph.get_tensor_by_name("initial_state:0")
    FinalStateTensor = loaded_graph.get_tensor_by_name("final_state:0")
    ProbsTensor = loaded_graph.get_tensor_by_name("probs:0")
    return (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [253]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    word_values = int_to_vocab.values()
    word_list = list(word_values)
#     print(probabilities)
#     print(int_to_vocab)
#     print(word_list)
#     print(int_to_vocab.values())
    return np.random.choice(list(int_to_vocab.values()), 1, p=probabilities)[0]

    #>>> aa_milne_arr = ['pooh', 'rabbit', 'piglet', 'Christopher']
    #>>> np.random.choice(aa_milne_arr, 5, p=[0.5, 0.1, 0.1, 0.3])
    #array(['pooh', 'pooh', 'pooh', 'Christopher', 'piglet'],
          #dtype='|S11')

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [254]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

moe_szyslak: boy, i'm too. i'm wasting it. burn into a place i barf that. just might stirring that?
barney_gumble:(on, friendly) umm. we can be" good man.
walther_hotenhoffer: that out. /
barney_gumble:(excited) oh, moe, i greedy beer?
c. _montgomery_burns: but then i have no choice.
homer_simpson:(brightening)
homer_simpson: help yourself rope in the beach. / the is gotta do?
bart_simpson: not no matter ever have my children!
moe_szyslak:(cool noises) yeah! homer, we go which if i didn't get myself it again. you're on!
seymour_skinner:(to book) someone's? one. no like myself!
moe_szyslak: you stole your daughter when i can just say, i rather think this calls will be got kids!
homer_simpson: that figures for guns.
lenny_leonard: yeah, someone ain't called heaven.
listen". wait is the field hair...
lenny_leonard: yeah, someone can called all right, i'm the kinda like


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.